# Modelos de Predicción
##### Futuras mejoras
* Estudiar bien los métodos de clasificación
* Cambiar el KNearestNeigh.. pues no está bien implementado: Falta normalizar, definir métrica, etc.
* Ordenar por tiempo y separar
* hacer crossvalidation
* 

## Importación de librerías

In [1]:
from os import path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

## Constantes

Rutas usuales que se ocuparán en el notebook

In [2]:
# Salvar gráficos
save_graf = False

# RUTAS
MAIN_PATH = path.join("..")

IMG_PATH = path.join(MAIN_PATH, "imagenes")
DATA_PATH = path.join(MAIN_PATH, "data")

WF_FOLDER_PATH = path.join(DATA_PATH,
                           "wildfires_us")
WF_DATA_PATH = path.join(WF_FOLDER_PATH,
                         "WILDFIRES_USA.csv")
WF_DATA_COLUMNS_PATH = path.join(WF_FOLDER_PATH,
                                 "WILDFIRES_USA_COLUMNS.csv")

# Tamaño de la imagen
my_figsize = (10, 5)

## Funciones auxiliares

Función que se ocupará para imprimir la información (número de filas y columnas) de un DataFrame.

In [3]:
def print_cantidad(dataframe):
    """Imprime la cantidad de datos que tiene el Data Frame.
    """
    msg_cantidad = "El dataset tiene una cantidad de {} datos y {} variables."
    print(msg_cantidad.format(dataframe.shape[0], dataframe.shape[1]))
    return None

## Carga de datos

### Columnas a ocupar

Se escojen las columnas a ocupar dependiendo de la importancia que tenga. Se omiten algunas columnas tales como las que son para el ID, como el nombre que tuvo el incendio, o la columna que indica de dónde se obtuvo el incendio; pues no deberían de afectar a la predicción.

In [4]:
# Todas las columnas
columnas = str(pd.read_csv(WF_DATA_COLUMNS_PATH).columns[0]).split(",")

# Columnas que se ocuparán en el análisis
columnas_ocupadas = columnas.copy()

# Columnas que no se ocuparán
columnas_sin_ocupar = [x for x in columnas if x not in columnas_ocupadas]

### Carga de Datos

In [5]:
df = pd.read_csv(WF_DATA_PATH)

# Convertimos los datos que sean fechas en ese tipo de dato
df["DISC_DATE_TIME"] = pd.to_datetime(df["DISC_DATE_TIME"])
df["CONT_DATE_TIME"] = pd.to_datetime(df["CONT_DATE_TIME"])
df["DT_FIRE"] = pd.to_timedelta(df["DT_FIRE"])

print_cantidad(df)

df.head()

El dataset tiene una cantidad de 255163 datos y 16 variables.


,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,DISC_DATE_TIME,DISC_MONTH,DISC_DOW,DISC_TIME,CONT_DATE_TIME,CONT_MONTH,CONT_DOW,CONT_TIME,DT_FIRE
0,2011,Fireworks,0.30,B,28.105500,-82.517700,FL,2011-01-01 00:01:00,January,Saturday,0,2011-01-01 03:35:00,January,Saturday,3,0 days 03:34:00
1,2011,Miscellaneous,1.00,B,38.198567,-93.002533,MO,2011-01-01 00:01:00,January,Saturday,0,2011-01-31 23:59:00,January,Monday,23,30 days 23:58:00
2,2011,Miscellaneous,0.10,A,32.927200,-115.127000,CA,2011-01-01 00:09:00,January,Saturday,0,2011-01-01 00:25:00,January,Saturday,0,0 days 00:16:00
3,2011,Children,0.18,A,33.187149,-84.918798,GA,2011-01-01 00:10:00,January,Saturday,0,2011-01-01 01:36:00,January,Saturday,1,0 days 01:26:00
4,2011,Children,0.01,A,40.068610,-77.118890,PA,2011-01-01 00:15:00,January,Saturday,0,2011-01-01 00:30:00,January,Saturday,0,0 days 00:15:00


## Preparación de la data

### Convertir variables categóricas a númericas

In [13]:
causas = df["STAT_CAUSE_DESCR"].unique().tolist()


In [15]:
# Copia del dataset
df_ = df.copy()
df_.head()

,FIRE_YEAR,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,DISC_DATE_TIME,DISC_MONTH,DISC_DOW,DISC_TIME,CONT_DATE_TIME,CONT_MONTH,CONT_DOW,CONT_TIME,DT_FIRE
0,2011,Fireworks,0.30,B,28.105500,-82.517700,FL,2011-01-01 00:01:00,January,Saturday,0,2011-01-01 03:35:00,January,Saturday,3,0 days 03:34:00
1,2011,Miscellaneous,1.00,B,38.198567,-93.002533,MO,2011-01-01 00:01:00,January,Saturday,0,2011-01-31 23:59:00,January,Monday,23,30 days 23:58:00
2,2011,Miscellaneous,0.10,A,32.927200,-115.127000,CA,2011-01-01 00:09:00,January,Saturday,0,2011-01-01 00:25:00,January,Saturday,0,0 days 00:16:00
3,2011,Children,0.18,A,33.187149,-84.918798,GA,2011-01-01 00:10:00,January,Saturday,0,2011-01-01 01:36:00,January,Saturday,1,0 days 01:26:00
4,2011,Children,0.01,A,40.068610,-77.118890,PA,2011-01-01 00:15:00,January,Saturday,0,2011-01-01 00:30:00,January,Saturday,0,0 days 00:15:00


In [14]:
columnas

['FIRE_YEAR',
 'STAT_CAUSE_DESCR',
 'FIRE_SIZE',
 'FIRE_SIZE_CLASS',
 'LATITUDE',
 'LONGITUDE',
 'STATE',
 'DISC_DATE_TIME',
 'DISC_MONTH',
 'DISC_DOW',
 'DISC_TIME',
 'CONT_DATE_TIME',
 'CONT_MONTH',
 'CONT_DOW',
 'CONT_TIME',
 'DT_FIRE']

In [16]:
# Columnas que no se ocuparán
drop_list = ["DISC_DATE_TIME",
             "CONT_DATE_TIME",
             "FIRE_SIZE_CLASS", 
             "DT_FIRE", 
             'FIRE_YEAR', 
             'CONT_MONTH',
             'CONT_DOW'
            ]

In [17]:
# Eliminamos las columnas que no se ocuparán
for to_drop in drop_list:
    df_ = df_.drop(to_drop, axis=1)
df_.head()

,STAT_CAUSE_DESCR,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,DISC_MONTH,DISC_DOW,DISC_TIME,CONT_TIME
0,Fireworks,0.30,28.105500,-82.517700,FL,January,Saturday,0,3
1,Miscellaneous,1.00,38.198567,-93.002533,MO,January,Saturday,0,23
2,Miscellaneous,0.10,32.927200,-115.127000,CA,January,Saturday,0,0
3,Children,0.18,33.187149,-84.918798,GA,January,Saturday,0,1
4,Children,0.01,40.068610,-77.118890,PA,January,Saturday,0,0


In [18]:
cols = [
    "STATE", 
    "DISC_MONTH",
    "DISC_DOW",
#     "CONT_MONTH",
#     "CONT_DOW",
#     "FIRE_SIZE_CLASS",
    "STAT_CAUSE_DESCR",
]

In [20]:

le = preprocessing.LabelEncoder()
for col in cols:
    df_[col] = le.fit_transform(df_[col])
    
df_.head()

,STAT_CAUSE_DESCR,FIRE_SIZE,LATITUDE,LONGITUDE,STATE,DISC_MONTH,DISC_DOW,DISC_TIME,CONT_TIME
0,5,0.30,28.105500,-82.517700,9,4,2,0,3
1,7,1.00,38.198567,-93.002533,24,4,2,0,23
2,7,0.10,32.927200,-115.127000,4,4,2,0,0
3,2,0.18,33.187149,-84.918798,10,4,2,0,1
4,2,0.01,40.068610,-77.118890,38,4,2,0,0


### Preparación de la data de entrenamiento con la de prueba

In [31]:
X = df_.drop("STAT_CAUSE_DESCR", axis=1) # .values
y = df.STAT_CAUSE_DESCR # .values
X.head(), y

(   FIRE_SIZE   LATITUDE   LONGITUDE  STATE  DISC_MONTH  DISC_DOW  DISC_TIME  \
 0       0.30  28.105500  -82.517700      9           4         2          0   
 1       1.00  38.198567  -93.002533     24           4         2          0   
 2       0.10  32.927200 -115.127000      4           4         2          0   
 3       0.18  33.187149  -84.918798     10           4         2          0   
 4       0.01  40.068610  -77.118890     38           4         2          0   
 
    CONT_TIME  
 0          3  
 1         23  
 2          0  
 3          1  
 4          0  ,
 0                 Fireworks
 1             Miscellaneous
 2             Miscellaneous
 3                  Children
 4                  Children
                 ...        
 255158        Miscellaneous
 255159                Arson
 255160    Missing/Undefined
 255161                Arson
 255162            Fireworks
 Name: STAT_CAUSE_DESCR, Length: 255163, dtype: object)

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.1,
                                                    random_state=0,
                                                    shuffle=False
                                                   )

In [33]:
# clases = list(y.unique())
# N = {c:len(y[y==c]) for c in clases}
# c_min = min(N, key=lambda x: N[x])
# w = {c:N[c_min]/N[c] for c in clases}

In [34]:
# from imblearn.under_sampling import NearMiss
# from collections import Counter

# us = NearMiss(n_neighbors=5, n_jobs=-1)#, sampling_strategy=w)
# X_us, y_us = us.fit_sample(X, y)
# X_us_train, X_us_test, y_us_train, y_us_test = train_test_split(
#     X_us,
#     y_us,
#     test_size=0.3,
#     random_state=0,
#     shuffle=False
# )

# print ("Distribution before resampling {}".format(Counter(y_train)))
# print ("Distribution after resampling {}".format(Counter(y_us_train)))

In [ ]:
# from sklearn.model_selection import TimeSeriesSplit

# tscv = TimeSeriesSplit()
# for train, test in tscv.split(X):
#     print(train)

## Generación de modelos

Empezamos a probar la clasificación con distintos métodos

### Naïve Bayes

Creamos un modelo y lo entrenamos

In [40]:
from sklearn.naive_bayes import GaussianNB
# Entrenamos
gnb = GaussianNB().fit(X_train, y_train)
gnb

GaussianNB()

Predecimos

In [36]:
# Predecimos
y_pred_gnb = gnb.predict(X_test)

Y mostramos los resultados

In [37]:
# Mostramos
print(classification_report(y_test, y_pred_gnb))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                   precision    recall  f1-score   support

            Arson       0.23      0.06      0.10      2148
         Campfire       0.00      0.00      0.00      1324
         Children       0.02      0.48      0.04       382
   Debris Burning       0.22      0.76      0.34      3683
    Equipment Use       0.00      0.00      0.00      1978
        Fireworks       0.10      0.27      0.15       428
        Lightning       0.75      0.07      0.12      6319
    Miscellaneous       0.15      0.00      0.00      5648
Missing/Undefined       0.19      0.10      0.13      2417
        Powerline       0.00      0.00      0.00       447
         Railroad       0.00      0.00      0.00        95
          Smoking       0.00      0.00      0.00       528
        Structure       0.00      0.00      0.00       120

         accuracy                           0.15     25517
        macro avg       0.13      0.13      0.07     25517
     weighted avg       0.29      0.15      0.10     2

### Clasificador de k-vecinos próximos

In [38]:
from sklearn.neighbors import KNeighborsClassifier
# Entrenamos
knc = KNeighborsClassifier(n_neighbors=5)
knc.fit(X_train, y_train)

# Predecimos
y_pred_knc = knc.predict(X_test)

# Mostramos
print(classification_report(y_test, y_pred_knc))

                   precision    recall  f1-score   support

            Arson       0.22      0.35      0.27      2148
         Campfire       0.19      0.26      0.22      1324
         Children       0.11      0.08      0.09       382
   Debris Burning       0.39      0.48      0.43      3683
    Equipment Use       0.23      0.13      0.16      1978
        Fireworks       0.37      0.29      0.32       428
        Lightning       0.65      0.70      0.67      6319
    Miscellaneous       0.43      0.41      0.42      5648
Missing/Undefined       0.50      0.37      0.42      2417
        Powerline       0.12      0.02      0.03       447
         Railroad       0.11      0.03      0.05        95
          Smoking       0.06      0.01      0.01       528
        Structure       0.09      0.01      0.02       120

         accuracy                           0.43     25517
        macro avg       0.27      0.24      0.24     25517
     weighted avg       0.42      0.43      0.42     2

### Análisis de Discriminante Lineal

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# Entrenamos
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# Predecimos
y_pred_lda = lda.predict(X_test)

# Mostramos
print(classification_report(y_test, y_pred_lda))

### Random Forest

In [27]:
from sklearn.ensemble import RandomForestClassifier
# Entrenamos
rfc = RandomForestClassifier(n_estimators=100, class_weight= "balanced", verbose=1, max_depth=5) # class_weight="balanced_subsample"
rfc = rfc.fit(X_train, y_train)

# Predecimos
y_pred_rfc = rfc.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   20.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished


In [28]:
# Mostramos
print(classification_report(y_test, y_pred_rfc))

                   precision    recall  f1-score   support

            Arson       0.46      0.19      0.27      3151
         Campfire       0.28      0.17      0.22      1399
         Children       0.13      0.17      0.15       670
   Debris Burning       0.46      0.49      0.48      6240
    Equipment Use       0.33      0.00      0.00      1666
        Fireworks       0.13      0.51      0.21       324
        Lightning       0.56      0.50      0.53      4502
    Miscellaneous       0.72      0.00      0.01      4768
Missing/Undefined       0.26      0.70      0.38      1664
        Powerline       0.07      0.17      0.09       355
         Railroad       0.05      0.21      0.08       189
          Smoking       0.06      0.33      0.11       488
        Structure       0.02      0.35      0.03       101

         accuracy                           0.31     25517
        macro avg       0.27      0.29      0.20     25517
     weighted avg       0.47      0.31      0.29     2

In [ ]:
# Entrenamos
rfc = RandomForestClassifier(n_estimators=50) # class_weight="balanced_subsample"
rfc = rfc.fit(X_train, y_train)

# Predecimos
y_pred_rfc = rfc.predict(X_test)

In [ ]:
# Mostramos
print(classification_report(y_test, y_pred_rfc))